# 引越しの口コミサイトをスクレイピング

参考：[複数のページをまたいでスクレイピングしたいです。](https://teratail.com/questions/220626?sort=3)

- サカイ引越センターの口コミ・評判(単身・家族)と見積もり料金相場
 - https://hikkoshizamurai.jp/company/d-sakaico/voice/p1-com101
 - moving_sq1.csv


- ハート引越センターの口コミ・評判(単身・家族)と見積もり料金相場
 - https://hikkoshizamurai.jp/company/d-hikkoshi8100/voice/p1-com50
 - moving_sq2.csv


- アリさんマークの引越社の口コミ・評判(単身・家族)と見積もり料金相場
 - https://hikkoshizamurai.jp/company/d-2626co/voice/p1-com3
 - moving_sq3.csv

- アーク引越センターの口コミ・評判(単身・家族)と見積もり料金相場
 - https://hikkoshizamurai.jp/company/d-0003co/voice/p1-com1
 - moving_sq4.csv

- アート引越センターの口コミ・評判(単身・家族)と見積もり料金相場
 - https://hikkoshizamurai.jp/company/d-the0123/voice/p1-com120
- moving_sq5.csv

In [32]:
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import datetime

date_data = []
compares_data = []
titles_data = []
scores_data = []
comments_data = []



for n in range(1, 250):
  time.sleep(1)
  url = f'https://hikkoshizamurai.jp/company/d-the0123/voice/p1-com120?voice_page={n}'
  res = requests.get(url)
  soup = BeautifulSoup(res.text,"html.parser")

  # 日付の取得
  tables = soup.find_all("table",attrs={"voice-table"})
  for i in range(len(tables)):
    date = tables[i].text.replace("\n","").replace("引越し時期","").replace("年","-").replace("月","-")
    dt, _ = date.split("日(")
    date_data.append(dt)


  # タイトルの取得
  titles = soup.find_all("div",attrs={"voice-title"})
  for t in titles:
    ts = t.text
    titles_data.append(ts)

  # 評価点の取得
  scores = soup.find_all("div",attrs={"voice-rate-number"})
  for s in scores:
    ss = s.text.replace("点","").replace("評価数：","").replace("\n", "").replace(" ","")
    scores_data.append(float(ss))

  # コメントの取得
  comments = soup.find_all("div",attrs={"voice-comment-text"})
  for c in comments:
    cs = c.text.replace("\n","").replace(" ","").replace("\r","")
    if len(cs) == 0:
      comments_data.append('none')
    else:
      comments_data.append(cs)



data = {"date":1,
        "title":1,
        "rate":1,
        "comment":1}

data["date"] = date_data
data["title"] = titles_data
data["rate"] = scores_data
data["comment"] = comments_data
df = pd.DataFrame(data)

なぜか

data["compare"] = compares_data

```
data["compare"] = compares_data
```
とすると



```
TypeError: 'str' object does not support item assignment
```
となってしまうが、別のセルで実行するとうまくいく


In [33]:
df.head(5)

,date,title,rate,comment
0,2023-8-27,アフターサービスも良かった,4.00,【見積もり時の対応】丁寧【料金】やや高め【作業内容】満足【全体的な評価】やや満足【良かった点...
1,2023-8-27,親切丁寧、まさにこの言葉がピッタリです！,5.00,悪かったことはありません。他の業者も何度か使いましたが、こんなに融通をきかせてくれたりはあり...
2,2023-8-29,2回目の利用でしたので安心感がありました。,4.75,3社と見積もりを比較して決定しました。大手3社の見積もりでしたが、1番アートさんが金額面と作...
3,2023-8-25,心配なことがたくさんありましたが丁寧な対応でした,5.00,引っ越しの条件が色々心配なことや初めての事がたくさんあり、お電話でいろんな質問をさせていただ...
4,2023-8-04,作業者の方がプロ対応でよかった,4.50,見積りは短時間で実施頂き、わかりやすく説明いただきました。荷物は多かったのですが初回提示で安...


- 参考：[Pythonのdatetimeで日付や時間と文字列を変換（strftime, strptime）](https://note.nkmk.me/python-datetime-usage/)

- 参考：[PythonでCSVオブジェクトを日付順で並べ替える
](https://laboratory.kazuuu.net/sorting-csv-objects-by-date-in-python/)

In [34]:
df["date"] = pd.to_datetime(df["date"])
df.sort_values(by = 'date', ascending = True, inplace = True)
df.head()

,date,title,rate,comment
2660,2018-04-10,テキパキ、丁寧。,4.00,急な引越しで、日にちもギリギリでしたが、なんとか配車いただき、値段もギリギリまで割引いていた...
490,2018-08-10,丁寧な応対,4.25,前回で二回目ですが、1回目は作業員の方の誘導が良く、今回は見積もりにきた方の対応が、非常に良...
4107,2018-11-03,大手で急な依頼でも対応してくれる。,4.50,引越しを依頼していた地元の業者と連絡が取れず、前日にドタキャン。代替えとして前日に依頼し、数...
3957,2019-03-30,信頼できる。,3.50,繁忙期だったので、対応が粗雑だった。引っ越し日は指定日以外ダメ、今の時点で了解を、もらわない...
4949,2019-05-12,あまり良くなかった,2.25,良かった点:手際が良かった悪かった点:手順と料金支払い方法が無かったので、後払いで良いと考え...


In [35]:
df.to_csv("moving5.csv")